# Wrapping a subset of a very large library

Sometimes, for a very large library, only a subset of available *C++* components is useful for end-users.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider only a subset of the *C++* components during the `Generate` step.
The **Clang** library is a complete *C*/*C++* compiler.
**Clang** is a great tool, but its stable *Python* interface (i.e. **libclang**) is lacking some useful features that are needed by **AutoWIG**.
In particular, class template specializations are not available in the abstract syntax tree.
Fortunately, most of the classes that would be needed during the traversal of the *C++* abstract syntax tree are not template specializations.
We therefore proposed to bootstrap the **Clang** *Python* bindings using the `libclang` `parser` of **AutoWIG**.
This new **Clang** *Python* interface is called **PyClangLite** and is able to parse class template specializations.
As for **libclang**, this interface is proposed only for a subset of the **Clang** library sufficient enough for proposing the new `pyclanglite` `parser`.

First, ensure that the **LLVM**/**Clang** technologies are installed on your computer and if there were built with `RTTI`.

In [1]:
import subprocess
subprocess.check_call(["llvm-config", "--version"])
subprocess.check_call(['clang++', '--version'])
if not subprocess.check_output(["llvm-config", "--has-rtti"]).strip() == 'YES':
    raise Exception('LLVM and Clang libraries have not been built with RTTI')

Note that these techonologies must be installed using the current system prefix

In [2]:
import sys
from path import path
prefix = path(sys.prefix).abspath()
if not prefix == subprocess.check_output(["llvm-config", "--prefix"]).strip():
    raise Exception('LLVM and Clang libraries have not been built with the \'' + prefix + '\' prefix')

In the following we will use **SCons** software construction tool for compiling *C++* libraries and their *Python* bindings.
This software is installed with the **Conda** package management system.

In [3]:
%%bash
conda install python-scons -c statiskit

Fetching package metadata .........
Solving package specifications: ..........

Package plan for installation in environment /home/main/miniconda:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python-scons-2.5.0         |           py27_0         739 KB  statiskit

The following NEW packages will be INSTALLED:

    python-scons: 2.5.0-py27_0 statiskit

Fetching packages ...
python-scons-2 100% |###############################| Time: 0:00:00 876.53 kB/s
Extracting packages ...
[      COMPLETE      ]|##################################################| 100%
Linking packages ...
[      COMPLETE      ]|##################################################| 100%


Using Anaconda API: https://api.anaconda.org


The **PyClangLite** GitHub repository must be cloned into the **PyClangLite** directory.

In [4]:
%%bash
git clone https://github.com/StatisKit/PyClangLite.git

Cloning into 'PyClangLite'...


This repository already has wrappers, we therefore need to remove them.

In [5]:
srcdir = path('PyClangLite')/'src'/'py'
for wrapper in srcdir.walkfiles('*.cpp'):
    wrapper.unlink()
for wrapper in srcdir.walkfiles('*.h'):
    wrapper.unlink()
wrapper = srcdir/'clanglite'/'_clanglite.py'
if wrapper.exists():
    wrapper.unlink()

In addition to the **Clang** libraries, the **ClangLite** library is needed in order to have access to some functionalities.
The `tool.h` header of this **ClangLite** library includes all necessary **Clang** headers.
This library is installed using the **SCons** `cpp`  target.

In [6]:
%%bash
scons cpp -C PyClangLite

scons: Entering directory `/home/main/AutoWIG/doc/examples/PyClangLite'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/cpp/tool.h" as "/home/main/miniconda/include/clanglite/tool.h"
g++ -o build/cpp/tool.os -c -std=c++0x -fvisibility-inlines-hidden -ffunction-sections -fdata-sections -Wno-deprecated-declarations -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -D_GNU_SOURCE -D__STDC_CONSTANT_MACROS -D__STDC_FORMAT_MACROS -D__STDC_LIMIT_MACROS -I/home/main/miniconda/include -I/home/main/miniconda/include/python2.7 build/cpp/tool.cpp
g++ -o /home/main/miniconda/lib/libclanglite.so -shared build/cpp/tool.os -L/home/main/miniconda/lib -lboost_python -lpython2.7 -lclangIndex -lclangARCMigrate -lclangRewriteFrontend -lclangFormat -lclangTooling -lclangToolingCore -lclangFrontend -lclangDriver -lclangSerialization -lclangParse -lclangSema -lclangStaticAnalyzerCheckers -lclangStaticAnalyzerCore -lclangRewrite -lclangAnalysis -lclangEd

Once these preliminaries done, we can proceed to the actual generation of wrappers for the **Clang** library.
For this, we import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

In [7]:
import autowig
asg = autowig.AbstractSemanticGraph()

We then parse the `tool.h` header of the **ClangLite** library with relevant compilation flags.

In [8]:
%%time
autowig.parser.plugin = 'libclang'
asg = autowig.parser(asg, [prefix/'include'/'clanglite'/'tool.h'],
               flags = ['-x', 'c++', '-std=c++11',
                        '-D__STDC_LIMIT_MACROS',
                        '-D__STDC_CONSTANT_MACROS',
                        '-I' + str((prefix/'include').abspath())],
               libpath = prefix/'lib'/'libclang.so',
               bootstrap = False,
               silent = True)

CPU times: user 2min 10s, sys: 436 ms, total: 2min 11s
Wall time: 2min 11s


Since most of **AutoWIG** guidelines are respected in the **Clang** library, the `default` `controller` implementation could be suitable.
Nevertheless, we need to force some *C++* components to be wrapped or not.
We therefore implements a new `controller`.

In [9]:
def clanglite_controller(asg):
    
    for node in asg['::boost::python'].classes(nested = True):
        node.is_copyable = True
        
    for node in asg.classes():
        node.boost_python_export = False
    for node in asg.enumerations():
        node.boost_python_export = False
    for node in asg.enumerators():
        if node.parent.boost_python_export:
            node.boost_python_export = False
    for node in asg.typedefs():
        node.boost_python_export = False
            
    from autowig.default_controller import refactoring
    asg = refactoring(asg)
    for fct in asg['::clanglite'].functions():
        if not fct.localname == 'build_ast_from_code_with_args':
            fct.parent = fct.parameters[0].qualified_type.desugared_type.unqualified_type
            
    for node in asg.functions(free = True):
        node.boost_python_export = False
    for node in asg.variables(free = True):
        node.boost_python_export = False
    
    asg['class ::boost::python::api::object'].boost_python_export = True
    asg['class ::boost::python::list'].boost_python_export = True 
    asg['class ::boost::python::str'].boost_python_export = True 

    subset = []
    classes = [asg['class ::clang::QualType'],
               asg['class ::clang::Type'],
               asg['class ::clang::Decl']]
    asg['class ::clang::QualType'].is_abstract = False
    asg['class ::clang::QualType'].is_copyable = True
    subset += classes
    for cls in classes:
        subset += cls.subclasses(recursive=True)
    subset.append(asg['class ::llvm::StringRef'])
    asg['class ::llvm::StringRef'].is_abstract = False
    asg['class ::llvm::StringRef'].is_copyable = True
    subset.append(asg['class ::clang::ASTUnit'])
    subset.append(asg['class ::clang::ASTContext'])
    subset.append(asg['class ::clang::SourceManager'])
    for mtd in asg['class ::clang::ASTContext'].methods(pattern='.*getSourceManager.*'):
        if mtd.return_type.globalname == 'class ::clang::SourceManager &':
                mtd.boost_python_export = True
                break
    subset.append(asg['class ::clang::FileID'])
    asg['class ::clang::FileID'].is_abstract = False
    asg['class ::clang::FileID'].is_copyable = True
    subset.append(asg['class ::clang::SourceLocation'])
    asg['class ::clang::SourceLocation'].is_abstract = False
    asg['class ::clang::SourceLocation'].is_copyable = True
    subset.append(asg['class ::clang::CXXBaseSpecifier'])
    subset.append(asg['class ::clang::DeclContext'])
    subset.append(asg['class ::clang::TemplateArgument'])
    asg['class ::clang::TemplateArgument'].is_abstract = False
    asg['class ::clang::TemplateArgument'].is_copyable = True
    subset.append(asg['class ::clang::TemplateArgumentList'])
    subset.append(asg['enum ::clang::Type::TypeClass'])
    subset.append(asg['enum ::clang::AccessSpecifier'])
    subset.append(asg['enum ::clang::LinkageSpecDecl::LanguageIDs'])
    subset.append(asg['enum ::clang::BuiltinType::Kind'])
    subset.append(asg['enum ::clang::TemplateArgument::ArgKind'])
    subset.append(asg['enum ::clang::Decl::Kind'])
    subset.extend(asg['::boost::python'].classes(nested = True))
    subset.extend(asg['::boost::python'].enumerations(nested = True))
    subset.extend(asg.nodes('::clanglite::build_ast_from_code_with_args'))

    for node in subset:
        node.boost_python_export = True

    if autowig.parser.plugin == 'libclang':
        for node in (asg.functions(pattern='.*(llvm|clang).*_(begin|end)')
                     + asg.functions(pattern='.*(llvm|clang).*getNameAsString')
                     + asg.nodes('::clang::NamedDecl::getQualifiedNameAsString')
                     + asg.nodes('::clang::ObjCProtocolDecl::collectInheritedProtocolProperties')
                     + asg.nodes('::clang::ASTUnit::LoadFromASTFile')
                     + asg.nodes('::clang::ASTUnit::getCachedCompletionTypes')
                     + asg.nodes('::clang::ASTUnit::getBufferForFile')
                     + asg.nodes('::clang::CXXRecordDecl::getCaptureFields')
                     + asg.nodes('::clang::ASTContext::SectionInfos')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::ASTContext::getObjCEncoding.*')
                     + asg.nodes('::clang::ASTContext::getAllocator')
                     + asg.nodes('::clang::QualType::getAsString')
                     + asg.nodes('::clang::SourceLocation::printToString')
                     + asg['class ::llvm::StringRef'].methods()):
            node.boost_python_export = False
            
    import sys
    from path import path
    for header in (path(sys.prefix)/'include'/'clang').walkfiles('*.h'):
        asg[header.abspath()].is_external_dependency = False
    
    return asg

This `controller` is then dynamically registered and used on the ASG.

In [10]:
%%time
autowig.controller['clanglite'] = clanglite_controller
autowig.controller.plugin = 'clanglite'
asg = autowig.controller(asg)

CPU times: user 1min 51s, sys: 112 ms, total: 1min 51s
Wall time: 1min 51s


In order to wrap a subset of the **Clang** library, we need to select the `boost_python_internal` `generator` implementation.

In [11]:
%%time
autowig.generator.plugin = 'boost_python_pattern'
wrappers = autowig.generator(asg,
                  module = srcdir/'_clanglite.cpp',
                  decorator = srcdir/'clanglite'/'_clanglite.py',
                  closure = False)

CPU times: user 8.17 s, sys: 40 ms, total: 8.21 s
Wall time: 8.18 s


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [12]:
%%time
wrappers.write()

CPU times: user 39.4 s, sys: 36 ms, total: 39.4 s
Wall time: 39.4 s


Here is an example of the generated wrappers.
We here present the wrappers for the `clang::Decl` class.

In [13]:
%%bash
pygmentize PyClangLite/src/py/wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98.cpp

#include "_clanglite.h"


namespace autowig
{
}


void wrapper_a6aedb4654a55a40aeecf4b1dc5fcc98()
{

    std::string name_7bbff48d109853e88270b3595c663a99 = boost::python::extract< std::string >(boost::python::scope().attr("__name__") + ".clang");
    boost::python::object module_7bbff48d109853e88270b3595c663a99(boost::python::handle<  >(boost::python::borrowed(PyImport_AddModule(name_7bbff48d109853e88270b3595c663a99.c_str()))));
    boost::python::scope().attr("clang") = module_7bbff48d109853e88270b3595c663a99;
    boost::python::scope scope_7bbff48d109853e88270b3595c663a99 = module_7bbff48d109853e88270b3595c663a99;
    void  (*method_pointer_4053c8f5abc2507ba73355adeb9b3769)() = ::clang::Decl::EnableStatistics;
    void  (*method_pointer_597650f0008559e89ad130c681eed082)() = ::clang::Decl::PrintStats;
    void  (*method_pointer_362dac5663ef50a8bfea193a64071d0d)(enum ::clang::Decl::Kind ) = ::clang::Decl::add;
    bool  (::clang::Decl::*method_pointer_395eb94f7fad57c391563c21c3de997a)

Once the wrappers are written on the disk, the bingings must be compiled and installed.
This can be done using the **SCons** `py`  target.